# TODO GO to KRM and create a volume first with name "volume-sentinel"

In krm create first a volume otherwise the RUN is stuck waiting for the volume
persisten_volume_claim specs:
- volume_name : volume-sentinel # this name is to be equal to the "name parameter inside volumes during the run of the function"
- space : 15Gbi
- storage_class_name: #disk
- access_mode: readWriteOnce

If you have any doubts on the params check the README.md inside https://github.com/tn-aixpa/sentinel-tools 

In [1]:
import digitalhub as dh
# conda install -c conda-forge gdal
PROJECT_NAME = "deforestation"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [2]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 2

In [ ]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2"
 },
 "startDate": "2018-01-01",
 "endDate": "2019-12-31",
 "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
 "area_sampling": "true",
 "cloudCover": "[0,20]",
 "artifact_name": "data_s2_deforestation" 
 }"""

list_args =  ["main.py",string_dict_data]
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.11.0_dev",command="python")

In [ ]:
run = function_s2.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    profile="1xa100",
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-deforestation",
        "mount_path": "/app/files",
        "spec": {
             "size": "350Gi" 
        }
    }])

In [ ]:
run.refresh().status.state

In [3]:
artifact = proj.get_artifact('data')

### Perform Data Elaboration on RS-Tool

In [ ]:
artifact_name='bosco'
src_path='bosco'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [7]:
artifact_name='data'
src_path='data'
artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [ ]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-deforestation:0.1_beta3", code_src="launch.sh")

In [9]:
function_rs = proj.get_function("elaborate")

In [ ]:
run_el = function_rs.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    profile="1xa100",
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-deforestation",
        "mount_path": "/app/files",
        "spec": { "size": "350Gi" }
    }
])

In [11]:
run_el.refresh().status.state

'RUNNING'

### Pipeline

In [ ]:
%%writefile "deforestation_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startYear, endYear, geometry, input1, input2, outputName):
    with pipeline_context() as pc:
        string_dict_data = """{"satelliteParams":{"satelliteType":"Sentinel2"},"startDate":\""""+ str(startYear) + """-01-01\","endDate": \"""" + str(endYear) + """-12-31\","geometry": \"""" + str(geometry) + """\","area_sampling":"true","cloudCover":"[0,20]","artifact_name":"data_s2_deforestation"}"""
        s1 = pc.step(name="download",
                     function="download_images_s2",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data],
                     profile="1xa100",
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-deforestation",
                        "mount_path": "/app/files",
                        "spec": { "size": "350Gi" }
                        }
                    ])
        s2 = pc.step(name="elaborate",
                     function="elaborate",
                     action="job",
                     fs_group='8877',
                     resources={"cpu": {"requests": "3"},"mem":{"requests": "16Gi"}},
                     profile="1xa100",
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-deforestation",
                        "mount_path": "/app/files",
                        "spec": { "size": "5Gi" }
                    }],
                     args=['/shared/launch.sh', str(input1), 'data_s2_deforestation', "[" + str(startYear) + ',' + str(endYear) + "]", str(outputName)]
                     ).after(s1)
     


Overwriting deforestation_pipeline.py


In [ ]:
workflow = proj.new_workflow(name="pipeline_deforestation", kind="kfp", code_src= "deforestation_pipeline.py", handler = "myhandler")

In [ ]:
wfbuild = workflow.run(action="build", wait=True)

2025-05-23 08:28:19,691 - INFO - Waiting for run 5eaa405fbae64195b4739f1db4571124 to finish...
2025-05-23 08:28:24,743 - INFO - Waiting for run 5eaa405fbae64195b4739f1db4571124 to finish...
2025-05-23 08:28:29,781 - INFO - Waiting for run 5eaa405fbae64195b4739f1db4571124 to finish...
2025-05-23 08:28:34,816 - INFO - Waiting for run 5eaa405fbae64195b4739f1db4571124 to finish...
2025-05-23 08:28:39,847 - INFO - Run 5eaa405fbae64195b4739f1db4571124 finished in 20.16 seconds.


{'kind': 'kfp+run', 'metadata': {'project': 'deforestation', 'name': '5eaa405fbae64195b4739f1db4571124', 'created': '2025-05-23T08:28:19.629Z', 'updated': '2025-05-23T08:28:32.078Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu', 'relationships': [{'type': 'run_of', 'dest': 'store://deforestation/workflow/kfp/pipeline_deforestation:7608fa0f3da8490fa4b7d2efa3697275'}]}, 'spec': {'task': 'kfp+build://deforestation/1bb1086e1718404591ae6cdf208e5246', 'local_execution': False, 'workflow': 'kfp://deforestation/pipeline_deforestation:7608fa0f3da8490fa4b7d2efa3697275', 'source': {'source': 'deforestation_pipeline.py', 'handler': 'myhandler', 'base64': 'CmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX2tmcC5kc2wgaW1wb3J0IHBpcGVsaW5lX2NvbnRleHQKCmRlZiBteWhhbmRsZXIoc3RhcnRZZWFyLCBlbmRZZWFyLCBnZW9tZXRyeSwgaW5wdXQxLCBpbnB1dDIsIG91dHB1dE5hbWUpOgogICAgd2l0aCBwaXBlbGluZV9jb250ZXh0KCkgYXMgcGM6CiAgICAgICAgc3RyaW5nX2RpY3RfZGF0YSA9ICIiInsic2F0ZWxsaXRlUGFyYW1zIjp7InNhdGVsbGl0ZVR5cGUiOiJTZW50aW5lbDIifSwi

In [ ]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startYear": 2018,
    "endYear": 2019,
    "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
    "input1":"bosco",
    'input2': "data_s2_deforestation",
    "outputName": "deforestation_2018_19"
    },
    wait=True)